# Scikit-learn Regression - Model Selection
작성자: 고정훈

Scikit-learn 홈페이지를 둘러보시면 정말 많은 Regression/Classification 알고리즘이 있습니다. 각 알고리즘은 저마다의 장단점이 있습니다. 데이터별로 궁합도 따져봐야 합니다. 하나의 알고리즘으로 모든 걸 해결 할 수 있다면 그 많은 알고리즘이 필요하지 않을 것입니다. 

알고리즘 선택이 끝이 아닙니다. 한 알고리즘 안에서도 Hyper-parameter를 어떻게 선택하느냐에 따라서 결과는 달라집니다. Neural Network를 사용하는 `MLPRegressor`은 hyper-parameter를 사용하는 것이 매우 중요합니다. 

끝이 아닙니다. 전처리(Preprocessing)을 어떻게 하느냐도 중요합니다. Min/Max scaling과 Standard Scaling 중 어느 쪽이 더 좋을지, Feature selection을 해야 하는지, 한다면 어떤 방법론을 써야 하는지, 차원 축소는 얼마나 해야할지 이 모든 것이 데이터에 따라 다르고, 함께 사용하는 학습 알고리즘에 따라 다릅니다. 

(전처리를 포함하여) 알고리즘을 무엇을 선택하고 hyper-parameter는 어떻게 선택할지 동시에 최적화 하는 문제를 CASH(Combined Algorithm Selection and Hyper-parameter optimization) Problem이라고 합니다. 

Model을 고를 수 있는 경우의 수가 많고, 성능과의 관계가 복잡하기 때문에, 많은 시도를 통해 찾아야 합니다. 이런 경우에 Baesian Optimization을 활용하면 좋긴하지만 Scikit-learn에서는 아직 지원하지 않습니다(2018년 4월 기준).

대신 Grid Search와 Random Search 방법을 제공합니다. Bayesian search는 병렬계산이 제한적이지만 Grid Search와 Random Search는 병렬화가 가능하기 때문에 자원이 풍부할 때는 충분히 좋은 결과를 만들어낼 수 있습니다. 다만 hyper-parameter optimization은 가능하지만 Algorithm Selection은 매뉴얼로 작업해야 합니다. 

## Grid Search

흔들어보고 싶은 hyper-parameter가 a, b라고 가정하겠습니다. a는 [1, 2, 3], b는 [10, 20, 30]을 각각 시도해보고 싶다면 Grid Search 는 a와 b의 가능한 모든 조합에 대해 다 시도해보는 것을 의미합니다. 

(1, 10), (2, 10), (3, 10), (1, 20), (2, 20), (3, 20), (1, 30), (2, 30), (3, 30)을 다 시도해보는 것이죠.

만약 흔들어 보고 싶은 hyper-parameter가 다수이고, 각각 많은 시도를 해보고 싶을 때는 그 조합 수가 무척 많아질 것입니다. Grid Search는 흔들 hyper-parameter 개수가 적거나, 한 hyper-parameter 내에서 흔들어 볼 후보 수가 적을 때 사용하는 것이 좋습니다. 

In [2]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor

x, y =load_diabetes(return_X_y=True)
y = np.reshape(y, (-1, 1))
x = StandardScaler().fit_transform(x)  
y = StandardScaler().fit_transform(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

regressor = MLPRegressor(max_iter=3000, random_state=2)  # Multi-layer perceptron을 정의합니다.

param_grid = {'hidden_layer_sizes': [(10,), (20,), (30,), (10, 10)],
              'solver': ['adam', 'lbfgs']
              }  # parameter를 흔들 space를 정의합니다.

gs = GridSearchCV(estimator=regressor, param_grid=param_grid, cv=KFold(shuffle=True), refit=True)  # Grid Search를 initialization 합니다.
gs.fit(x_train, y_train.ravel())  # hyper-parameter를 흔들어가며 최적의 hyper-parameter를 찾습니다. refit=True이기 때문에 찾은 모델에 대해 전체 데이터로 다시 학습합니다.  
score = gs.score(x_test, y_test.ravel())  # Test score를 계산해봅니다. 
print('Score:', score)  

Score: 0.372839693968


`GridSearchCV`는 *param_grid*에 입력된 parameter space의 모든 조합에 대해 내부적으로 cross validation을 진행한 후 가장 성능이 좋은 hyper-parameter를 찾아 전체 데이터에 대해서 다시 학습합니다. 

어떤 hyper-parameter 조합을 최적으로 찾았는지 볼까요?

In [3]:
print(gs.best_params_)

{'hidden_layer_sizes': (30,), 'solver': 'adam'}


`GridSearchCV`로 `predict`, `score`등 estimator가 수행하는 method를 그대로 사용할 수 있습니다. 가장 성능이 좋은 모델을 기준으로 수행하는 것이죠. 만약 best model을 끄집어내고 싶으시다면 아래와 같이 하시면 됩니다. 

In [4]:
regressor = gs.best_estimator_
print(regressor)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(30,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=3000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=2, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)


자세한 탐색 결과를 알고 싶다면 아래 attribute를 불러오면 됩니다. 

In [5]:
import pandas as pd
cv_result = pd.DataFrame(gs.cv_results_)
print(cv_result)

   mean_fit_time  mean_score_time  mean_test_score  mean_train_score  \
0       0.168785         0.000000         0.488294          0.569908   
1       0.457323         0.000000         0.079410          0.832664   
2       0.113080         0.000334         0.493556          0.610789   
3       0.973021         0.000000        -1.202203          0.992400   
4       0.159111         0.000668         0.499207          0.638202   
5       0.458325         0.000000        -1.370984          0.999989   
6       0.185798         0.000334         0.430692          0.604212   
7       1.010047         0.000000        -0.959507          0.954842   

  param_hidden_layer_sizes param_solver  \
0                    (10,)         adam   
1                    (10,)        lbfgs   
2                    (20,)         adam   
3                    (20,)        lbfgs   
4                    (30,)         adam   
5                    (30,)        lbfgs   
6                 (10, 10)         adam   
7      

## Random Search

Random Search는 탐색 공간(search space)에서 무작위로 hyper-parameter를 추출합니다. Grid Search가 정해놓은 값 중들의 조합을 모두 시도한다면, Random Search는 정해준 구간 내에서 정해준 시도 횟수만큼 hyper-parameter를 테스트합니다. 

Random Search의 장점은 시도해볼 hyper-parameter가 많아도 search가 가능하며, <U>의외로 결과가 좋다는 것입니다</U>. 얼핏 생각하면 무작위로 hyper-parameter 조합을 만들면 최적을 우연히 찾기 쉽지 않을 것 같습니다. 한 가지 정답이 있는 것이 아니라 다양한 조합이 최적에 가까운(sub-optimal) 결과를 만들어내기 때문에 random search를 통해서도 의미있는 결과를 만들어 낼 수 있습니다. 

Random Search와 대변되는 것이 Bayesian Optimization입니다. Bayesian은 sequential하게 다음 모델을 고르기 때문에 병렬화가 불가능하지만,  Random Search는 서로가 무관한 trial이기 때문에 하드웨어가 허락하는만큼 동시에 여러 모델을 시도해볼 수 있는 장점 또한 있습니다. 

Scikit-learn에서는 RandomizedSearchCV를 이용해 random search를 할 수 있습니다. 사용법은 GridSearchCV와 거의 유사하나 parameter space를 만드는 방법만 다릅니다. 

RandomizedSearchCV에서는 GridSearchCV에서처럼 list안에 후보 변수들을 입력할 필요 없습니다. 특정 분포로부터 뽑아내게 할 수 있습니다. 

In [11]:
from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV


param_dist = {'hidden_layer_sizes': [(n_node,) for n_node in range(5, 31)],
              'solver': ['adam', 'lbfgs'],
              'alpha': uniform(0.0001, 0.005),
              'max_iter': randint(1000, 5001),
              } # parameter를 흔들 space를 정의합니다.

rs = RandomizedSearchCV(estimator=regressor, 
                        param_distributions=param_dist, 
                        n_iter=30,
                        cv=KFold(shuffle=True), 
                        refit=True,
                        random_state=2)  # Random Search를 initialization 합니다.
rs.fit(x_train, y_train.ravel())  # hyper-parameter를 흔들어가며 최적의 hyper-parameter를 찾습니다. refit=True이기 때문에 찾은 모델에 대해 전체 데이터로 다시 학습합니다.  
score = rs.score(x_test, y_test.ravel())  # Test score를 계산해봅니다. 
print('Score:', score)  

Score: 0.390698125197


In [12]:
print(rs.best_params_)

{'hidden_layer_sizes': (14,), 'alpha': 0.00069742399164656518, 'max_iter': 1366, 'solver': 'adam'}


In [13]:
cv_result = pd.DataFrame(rs.cv_results_)
print(cv_result)

    mean_fit_time  mean_score_time  mean_test_score  mean_train_score  \
0        0.178464         0.000334         0.501589          0.597321   
1        0.991696         0.000667        -0.329965          0.933950   
2        0.601091         0.000000        -0.452470          0.929060   
3        0.121747         0.000334         0.445352          0.601867   
4        0.220159         0.000333         0.468094          0.558121   
5        0.116074         0.000000         0.481407          0.603741   
6        0.226826         0.000334         0.468138          0.558137   
7        0.621439         0.000333        -0.203778          0.878229   
8        0.208147         0.000333         0.484295          0.568705   
9        0.099404         0.000000         0.497956          0.607318   
10       0.739522         0.000667        -1.009655          0.994912   
11       0.098736         0.000667         0.497941          0.607310   
12       0.468331         0.000000        -1.090018

## Pipeline-Search 결합

학습 알고리즘에도 Hyper-parameter가 있지만 Preprocessing에도 있습니다. 가령 MinMaxScaler는 minimum과 maximum을 어떻게 설정할지 정할 수 있습니다. 차원을 축소하는 PCA에서는 차원을 얼마로 축소할지 결정하는 것이 hyper-parameter 중 하나입니다. 

Preprocessing에서 사용하는 hyper-parameter는 학습에서 사용하는 hyper-parameter와 결합되어서 최적화 해야 합니다. 전체 Workflow를 최적화해야 하는 것이죠. 

Pipeline과 GridSearchCV, RandomizedSearchCV를 결합해서 사용하면 가능합니다. `Pipeline` Tutorial에서 pipeline으로 구성하면 하나의 operator처럼 동작한다고 했습니다.

In [14]:
import numpy as np
from scipy.stats import uniform, randint
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

x, y =load_diabetes(return_X_y=True)
y = np.reshape(y, (-1, 1))
y_scaler = MinMaxScaler(feature_range=(-1, 1))
y_scaled = y_scaler.fit_transform(y)  # y는 미리 scaling 합니다. y scaling은 pipeline에 넣지 않습니다. 

x_train, x_test, y_train, y_test = train_test_split(x, y_scaled, random_state=1)

x_scaler = MinMaxScaler()


pca = PCA()  # PCA instance를 만듭니다. 본 예제에서 PCA는 큰 효과를 주지 않지만 예시를 위해 추가했습니다. 

regressor = MLPRegressor(max_iter=3000, random_state=2)  # Multi-layer perceptron을 정의합니다.

steps = [('scaler', x_scaler), ('pca', pca), ('regressor', regressor)]
pipeline = Pipeline(steps)

param_dist = {'scaler__feature_range': [(-1, 1), (0, 1)],
              'pca__n_components': randint(7, x_train.shape[1]+1),
              'regressor__hidden_layer_sizes': [(n_node,) for n_node in range(5, 31)],
              'regressor__solver': ['adam', 'lbfgs'],
              'regressor__alpha': uniform(0.0001, 0.005),
              'regressor__max_iter': randint(1000, 5001),
              } # parameter를 흔들 space를 정의합니다. step 이름과 parameter 이름을 '__'로 붙입니다. 


rs = RandomizedSearchCV(estimator=pipeline, 
                        param_distributions=param_dist, 
                        n_iter=50,
                        cv=KFold(shuffle=True), 
                        refit=True,
                        random_state=3)  # Random Search를 initialization 합니다.
rs.fit(x_train, y_train.ravel())  # hyper-parameter를 흔들어가며 최적의 hyper-parameter를 찾습니다. refit=True이기 때문에 찾은 모델에 대해 전체 데이터로 다시 학습합니다.  
score = rs.score(x_test, y_test.ravel())  # Test score를 계산해봅니다. 
print('Score:', score)  

Score: 0.403329016652


In [16]:
print(rs.best_params_)

{'regressor__alpha': 0.0045886851489163301, 'scaler__feature_range': (0, 1), 'regressor__solver': 'adam', 'regressor__max_iter': 2023, 'pca__n_components': 10, 'regressor__hidden_layer_sizes': (9,)}


pipeline 내부의 operator의 hyper-parameter에 접근하려면 조금 독특한 방법을 써야 합니다. step의 이름과 해당 opeartor의 hyper-parameter를 underscore 두개 '\_\_'로 연결해야 합니다. param_distribution에 입력할 때도 동일하게 입력해야 합니다. 다른 건 operator 한 개에 대한 예시와 동일합니다.  

Pipeline과 Model Search 기능을 잘 사용하면 성능이 높은 Workflow를 손쉽게 만들 수 있습니다. 